In [2]:
"""
evaluate.py
- Load pipeline and dataset, produce classification report & confusion matrix figure.

This version is safe to import/run inside Jupyter: it exposes an evaluate() function,
uses parse_known_args() to tolerate extra argv injected by IPython, and does not
automatically try to parse CLI args when run inside a notebook kernel.
"""
import argparse
import os
import sys
import pandas as pd
import joblib
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def evaluate(model_path: str, input_csv: str, out_dir: str = "social-media-sentiment-analysis/figures"):
    """
    Load model at model_path, read processed csv at input_csv (must contain text_clean and label),
    print classification report and save a confusion matrix png into out_dir.
    """
    # Basic validations
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found: {model_path}")
    if not os.path.exists(input_csv):
        raise FileNotFoundError(f"Input CSV not found: {input_csv}")

    # Load model
    model = joblib.load(model_path)

    # Read data
    df = pd.read_csv(input_csv, encoding="utf-8")
    if "text_clean" not in df.columns:
        raise SystemExit("Input CSV must contain a 'text_clean' column")
    if "label" not in df.columns:
        raise SystemExit("Input CSV must contain a 'label' column")

    X = df["text_clean"].astype(str)
    y = df["label"].astype(str)

    # Predict
    preds = model.predict(X)

    # Classification report
    print("Classification Report:")
    print(classification_report(y, preds))

    # Confusion matrix - ensure labels include union of true/pred to avoid mismatch
    labels = sorted(set(list(y) + list(preds)))
    cm = confusion_matrix(y, preds, labels=labels)

    # Ensure out directory exists
    os.makedirs(out_dir, exist_ok=True)

    # Plot heatmap
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=labels, yticklabels=labels, cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    p = os.path.join(out_dir, "confusion_matrix.png")
    plt.tight_layout()
    plt.savefig(p)
    plt.close()
    print("Saved confusion matrix to", p)
    return {"report": classification_report(y, preds, output_dict=True), "confusion_matrix_path": p}

def main(argv=None):
    parser = argparse.ArgumentParser(prog="evaluate.py")
    parser.add_argument("--model", required=True, help="path to saved model pipeline (joblib)")
    parser.add_argument("--input", required=True, help="processed csv with text_clean and label")
    parser.add_argument("--out-dir", default="social-media-sentiment-analysis/figures")
    # use parse_known_args so running inside Jupyter/IPython (which injects extra args) won't fail
    if argv is None:
        args, unknown = parser.parse_known_args()
    else:
        args, unknown = parser.parse_known_args(argv)

    evaluate(args.model, args.input, args.out_dir)

if __name__ == "__main__":
    # Detect interactive environment (Jupyter/IPython). If interactive, do not auto-run CLI parsing.
    in_ipy = False
    try:
        from IPython import get_ipython  # type: ignore
        if get_ipython() is not None:
            in_ipy = True
    except Exception:
        in_ipy = False

    if in_ipy:
        msg = (
            "Detected IPython/Jupyter environment. To run evaluation here:\n"
            "  - Import and call evaluate() directly, for example:\n"
            "      from evaluate import evaluate\n"
            "      evaluate('path/to/model.joblib', 'data/processed/tweets_clean.csv')\n\n"
            "Or run this script from a terminal:\n"
            "  python evaluate.py --model path/to/model.joblib --input data/processed/tweets_clean.csv --out-dir results/figures\n"
        )
        print(msg)
    else:
        main()

Detected IPython/Jupyter environment. To run evaluation here:
  - Import and call evaluate() directly, for example:
      from evaluate import evaluate
      evaluate('path/to/model.joblib', 'data/processed/tweets_clean.csv')

Or run this script from a terminal:
  python evaluate.py --model path/to/model.joblib --input data/processed/tweets_clean.csv --out-dir results/figures

